# Landsat - Extraction des Données Satellitaires

## C'est quoi Landsat ?

**Landsat** est un programme de satellites d'observation de la Terre (NASA/USGS) :
- Des images depuis **1972** (le plus ancien programme satellite civil)
- Une résolution spatiale de **30 mètres**
- Des images tous les **16 jours** pour un même lieu
- Plusieurs **bandes spectrales** (visible, infrarouge, etc.)

## Pourquoi on en a besoin ?

Les images satellite permettent de **voir** l'état de l'eau et de son environnement :
- **Bande verte (Green)** → Détecte la turbidité et les algues
- **Bande NIR (proche infrarouge)** → Distingue l'eau de la végétation
- **Bandes SWIR (infrarouge à ondes courtes)** → Détecte l'humidité et les minéraux

## Ce que fait ce notebook

```
1. Se connecter à l'API Microsoft Planetary Computer
         ↓
2. Pour chaque site de mesure d'eau :
   - Chercher l'image Landsat la plus proche en date
   - Filtrer les images avec peu de nuages (<10%)
   - Extraire les valeurs spectrales (buffer de 100m)
         ↓
3. Calculer les indices spectraux (NDMI, MNDWI)
         ↓
4. Sauvegarder le fichier CSV avec les features Landsat
```

## Différence avec TerraClimate

| Aspect | TerraClimate | Landsat |
|--------|--------------|----------|
| Type | Données climatiques (grille) | Images satellite (scènes) |
| Format | Zarr (chargement rapide) | Scènes individuelles (API) |
| Résolution | ~4 km | 30 m |
| Temps d'exécution | ~5 minutes | ~2-7 heures |

⚠️ **Note importante** : Contrairement à TerraClimate, Landsat nécessite des appels API individuels pour chaque point. Le traitement est donc plus lent mais peut être parallélisé.

## Source des données

Documentation : [Landsat Collection 2 Level 2 sur Planetary Computer](https://planetarycomputer.microsoft.com/dataset/landsat-c2-l2)

---

## Étape 1 : Installation des dépendances

**Première exécution uniquement** : Après avoir exécuté cette cellule, il faut redémarrer le kernel :
1. Cliquer sur "Connected" en haut
2. Sélectionner "Restart kernel"

Les exécutions suivantes n'ont pas besoin de ce redémarrage.

In [1]:
!pip install uv
!uv pip install --system -r ../requirements.txt

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip
Using Python 3.13.1 environment at: c:\Program Files\Python313
Resolved 194 packages in 6.14s
 Downloaded shapely
 Downloaded pywinpty
 Downloaded widgetsnbextension
 Downloaded xarray
 Downloaded pydap
 Downloaded pygments
 Downloaded dask
 Downloaded lxml
 Downloaded debugpy
 Downloaded matplotlib
 Downloaded scikit-learn
 Downloaded babel
 Downloaded jupyterlab
 Downloaded netcdf4
 Downloaded notebook
 Downloaded rasterio
 Downloaded pandas
 Downloaded jedi
Prepared 98 packages in 2m 00s
error: Failed to install: arrow-1.4.0-py3-none-any.whl (arrow==1.4.0)
  Caused by: failed to create directory `c:\Program Files\Python313\Lib\site-packages\arrow`: Accès refusé. (os error 5)


In [3]:
!pip install odc-stac

Defaulting to user installation because normal site-packages is not writeable
  Using cached affine-2.4.0-py3-none-any.whl.metadata (4.0 kB)
  Using cached cligj-0.7.2-py3-none-any.whl.metadata (5.0 kB)
   ---------------------------------------- 0.0/30.1 MB ? eta -:--:--
   ------ --------------------------------- 4.7/30.1 MB 23.5 MB/s eta 0:00:02
   ------------ --------------------------- 9.4/30.1 MB 22.5 MB/s eta 0:00:01
   ------------------- -------------------- 14.7/30.1 MB 22.8 MB/s eta 0:00:01
   -------------------------- ------------- 19.7/30.1 MB 22.8 MB/s eta 0:00:01
   --------------------------------- ------ 25.2/30.1 MB 22.8 MB/s eta 0:00:01
   ---------------------------------------  29.9/30.1 MB 22.8 MB/s eta 0:00:01
   ---------------------------------------- 30.1/30.1 MB 20.6 MB/s eta 0:00:00
Using cached affine-2.4.0-py3-none-any.whl (15 kB)
Using cached cligj-0.7.2-py3-none-any.whl (7.1 kB)
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   --


[notice] A new release of pip is available: 24.3.1 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# =============================================================================
# IMPORTS
# =============================================================================

# Snowflake (uniquement si le notebook tourne dans Snowflake Notebooks)
# Commenter ces lignes pour exécuter en local
# import snowflake
# from snowflake.snowpark.context import get_active_session
# session = get_active_session()

import warnings
warnings.filterwarnings("ignore")

# Manipulation de données
import numpy as np
import pandas as pd

# Accès à l'API Microsoft Planetary Computer
import pystac_client      # Pour naviguer dans le catalogue de données
import planetary_computer as pc  # Pour s'authentifier
from odc.stac import stac_load  # Pour charger les données satellite

# Traitement parallèle (pour accélérer l'extraction)
from concurrent.futures import ThreadPoolExecutor, as_completed

from datetime import date
from tqdm import tqdm  # Barre de progression
import os

print("Imports OK!")

Imports OK!


---

## Étape 2 : Définition des fonctions

On va créer 3 fonctions :

| Fonction | Rôle |
|----------|------|
| `extract_landsat_for_point()` | Extrait les valeurs Landsat pour un point donné |
| `extract_landsat_batch()` | Traite un lot de points en parallèle |
| `compute_spectral_indices()` | Calcule NDMI et MNDWI à partir des bandes |

### Comprendre les bandes Landsat

Landsat capture la lumière à différentes longueurs d'onde (bandes spectrales) :

```
Spectre électromagnétique :

    Visible                    Infrarouge
    ├────────────────────┼─────────────────────────────────┤
    │  Blue  Green  Red  │  NIR     SWIR16     SWIR22      │
    │   🔵    🟢    🔴   │   🟤       ⬛         ⬛         │
    └────────────────────┴─────────────────────────────────┘
     0.4    0.5   0.6    0.8      1.6        2.2    (μm)
```

**Bandes utilisées pour la qualité de l'eau :**

| Bande | Nom | Utilité |
|-------|-----|----------|
| **green** | Vert (0.53-0.59 μm) | Pénètre l'eau, détecte turbidité/algues |
| **nir08** | Proche infrarouge (0.85-0.88 μm) | Absorbé par l'eau, détecte végétation |
| **swir16** | SWIR 1 (1.57-1.65 μm) | Détecte humidité du sol et végétation |
| **swir22** | SWIR 2 (2.11-2.29 μm) | Distingue minéraux et types de sol |

### Pourquoi un buffer de 100 mètres ?

On ne prend pas juste le pixel exact du site de mesure, mais une zone de **100m autour** :

```
    Sans buffer (1 pixel)          Avec buffer 100m (zone 3x3)
    ┌─────┐                         ┌─────┬─────┬─────┐
    │  •  │  30m                    │     │     │     │
    └─────┘                         ├─────┼─────┼─────┤
                                    │     │  •  │     │
                                    ├─────┼─────┼─────┤
                                    │     │     │     │
                                    └─────┴─────┴─────┘
                                           ~100m
```

**Avantages du buffer :**
1. **Réduit le bruit** : Un seul pixel peut être affecté par des erreurs
2. **Médiane plus robuste** : On prend la médiane des pixels, moins sensible aux outliers
3. **Correspond à la réalité** : L'eau mesurée n'est pas un point mais une zone

**Calcul du buffer en degrés :**
- 1 degré ≈ 111 km à l'équateur
- 100m ÷ 111000m = **0.0009 degrés**

### Schéma du processus d'extraction

Pour chaque site de mesure, voici ce qui se passe :

```
Site de mesure                    Catalogue Planetary Computer
┌────────────────────┐            ┌─────────────────────────────┐
│ Lat: -28.76        │   Requête  │ Scènes Landsat disponibles: │
│ Lon: 17.73         │ ──────────►│ • 2011-01-05 (nuages: 45%) ❌│
│ Date: 02-01-2011   │            │ • 2011-01-15 (nuages: 5%)  ✓│
└────────────────────┘            │ • 2011-01-21 (nuages: 12%) ❌│
                                  └─────────────────────────────┘
                                              │
                                              ▼
                              Sélection : scène la plus proche
                              avec nuages < 10%
                                              │
                                              ▼
                              ┌─────────────────────────────┐
                              │ Extraction des bandes :     │
                              │ • green: 0.142 (médiane)    │
                              │ • nir08: 0.231 (médiane)    │
                              │ • swir16: 0.187 (médiane)   │
                              │ • swir22: 0.098 (médiane)   │
                              └─────────────────────────────┘
```

In [5]:
# =============================================================================
# CONFIGURATION
# =============================================================================

# Buffer de ~100m autour de chaque point (en degrés)
BUFFER_DEG = 0.0009

# Bandes spectrales à extraire
BANDS = ["green", "nir08", "swir16", "swir22"]

# Période de recherche des données
DATE_RANGE = "2011-01-01/2015-12-31"

# Seuil de couverture nuageuse maximum (%)
MAX_CLOUD_COVER = 10

# Nombre de workers pour le traitement parallèle
N_WORKERS = 4

# Dossier de sortie
OUTPUT_DIR = "../data/processed"

In [6]:
def get_catalog():
    """
    Initialise la connexion au catalogue Planetary Computer.
    
    Retourne :
        pystac_client.Client : Client connecté au catalogue
    """
    return pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1",
        modifier=pc.sign_inplace,
    )

In [7]:
def extract_landsat_for_point(lat, lon, sample_date, catalog=None):
    """
    Extrait les valeurs Landsat pour un point géographique donné.
    
    Paramètres :
        lat : float - Latitude du site
        lon : float - Longitude du site
        sample_date : str ou datetime - Date de l'échantillon
        catalog : pystac_client.Client - Catalogue (optionnel, créé si non fourni)
    
    Retourne :
        dict : Valeurs médianes pour chaque bande (ou NaN si échec)
    
    Processus :
        1. Créer une bounding box de ~100m autour du point
        2. Chercher les scènes Landsat avec peu de nuages
        3. Sélectionner la scène la plus proche de la date
        4. Extraire la médiane des pixels dans la bounding box
    """
    
    # Résultat par défaut en cas d'échec
    default_result = {band: np.nan for band in BANDS}
    
    try:
        # Convertir la date
        if isinstance(sample_date, str):
            sample_date = pd.to_datetime(sample_date, dayfirst=True, errors='coerce')
        
        if pd.isna(sample_date):
            return default_result
        
        # Créer la bounding box (~100m autour du point)
        bbox = [
            lon - BUFFER_DEG / 2,  # min_lon
            lat - BUFFER_DEG / 2,  # min_lat
            lon + BUFFER_DEG / 2,  # max_lon
            lat + BUFFER_DEG / 2   # max_lat
        ]
        
        # Initialiser le catalogue si non fourni
        if catalog is None:
            catalog = get_catalog()
        
        # Rechercher les scènes Landsat
        search = catalog.search(
            collections=["landsat-c2-l2"],
            bbox=bbox,
            datetime=DATE_RANGE,
            query={"eo:cloud_cover": {"lt": MAX_CLOUD_COVER}},
        )
        
        items = search.item_collection()
        
        # Pas de scène disponible
        if not items:
            return default_result
        
        # Convertir la date en UTC pour comparaison
        sample_date_utc = sample_date.tz_localize("UTC") if sample_date.tzinfo is None else sample_date.tz_convert("UTC")
        
        # Trier par proximité à la date d'échantillonnage
        items_sorted = sorted(
            items,
            key=lambda x: abs(pd.to_datetime(x.properties["datetime"]).tz_convert("UTC") - sample_date_utc)
        )
        
        # Sélectionner la scène la plus proche
        selected_item = pc.sign(items_sorted[0])
        
        # Charger les bandes
        data = stac_load([selected_item], bands=BANDS, bbox=bbox).isel(time=0)
        
        # Calculer les médianes
        result = {}
        for band in BANDS:
            median_val = float(data[band].astype("float").median(skipna=True).values)
            # Remplacer 0 par NaN (valeur invalide)
            result[band] = median_val if median_val != 0 else np.nan
        
        return result
        
    except Exception as e:
        return default_result

In [8]:
def extract_landsat_batch(df, n_workers=N_WORKERS):
    """
    Extrait les valeurs Landsat pour tous les points d'un DataFrame.
    Utilise le traitement parallèle pour accélérer l'extraction.
    
    Paramètres :
        df : DataFrame - Doit contenir 'Latitude', 'Longitude', 'Sample Date'
        n_workers : int - Nombre de threads parallèles
    
    Retourne :
        DataFrame : Valeurs Landsat pour chaque point
    
    Note :
        Le traitement parallèle accélère significativement l'extraction
        car les appels API sont I/O-bound (attente réseau).
    """
    
    results = []
    
    # Préparer les données
    points = list(df[['Latitude', 'Longitude', 'Sample Date']].itertuples(index=False))
    
    # Traitement parallèle avec ThreadPoolExecutor
    with ThreadPoolExecutor(max_workers=n_workers) as executor:
        # Soumettre toutes les tâches
        future_to_idx = {
            executor.submit(extract_landsat_for_point, lat, lon, date): idx
            for idx, (lat, lon, date) in enumerate(points)
        }
        
        # Initialiser les résultats avec des NaN
        results = [{band: np.nan for band in BANDS} for _ in range(len(points))]
        
        # Collecter les résultats avec barre de progression
        for future in tqdm(as_completed(future_to_idx), total=len(points), desc="Extraction Landsat"):
            idx = future_to_idx[future]
            try:
                results[idx] = future.result()
            except Exception as e:
                pass  # Garde les NaN par défaut
    
    return pd.DataFrame(results)

In [9]:
def compute_spectral_indices(df):
    """
    Calcule les indices spectraux à partir des bandes Landsat.
    
    Paramètres :
        df : DataFrame - Doit contenir 'nir08', 'green', 'swir16'
    
    Retourne :
        DataFrame : Avec colonnes NDMI et MNDWI ajoutées
    
    Indices calculés :
        - NDMI : Normalized Difference Moisture Index
        - MNDWI : Modified Normalized Difference Water Index
    """
    
    df = df.copy()
    
    # Epsilon pour éviter la division par zéro
    eps = 1e-10
    
    # NDMI = (NIR - SWIR16) / (NIR + SWIR16)
    # Mesure le contenu en eau de la végétation
    df['NDMI'] = (df['nir08'] - df['swir16']) / (df['nir08'] + df['swir16'] + eps)
    
    # MNDWI = (Green - SWIR16) / (Green + SWIR16)
    # Détecte les surfaces d'eau libre
    df['MNDWI'] = (df['green'] - df['swir16']) / (df['green'] + df['swir16'] + eps)
    
    return df

### Indices spectraux calculés

À partir des bandes brutes, on calcule deux indices utiles pour la qualité de l'eau :

**NDMI (Normalized Difference Moisture Index)**
```
NDMI = (NIR - SWIR16) / (NIR + SWIR16)
```
- Mesure le **contenu en eau** de la végétation et du sol
- Valeurs hautes (0.4 à 1.0) → zones humides, végétation bien irriguée
- Valeurs basses (-1.0 à 0.0) → sol sec, stress hydrique

**MNDWI (Modified Normalized Difference Water Index)**
```
MNDWI = (Green - SWIR16) / (Green + SWIR16)
```
- Détecte les **surfaces d'eau libre**
- Valeurs positives → présence d'eau
- Valeurs négatives → sol ou végétation

```
Interprétation visuelle :

NDMI :  -1.0 ◄────────────────────────────► +1.0
         🏜️ Sec                      Humide 💧

MNDWI : -1.0 ◄────────────────────────────► +1.0
         🌿 Végétation                  Eau 🌊
```

---

## Étape 3 : Extraction pour les données d'entraînement

### Variables Landsat extraites

| Variable | Description | Lien avec la qualité de l'eau |
|----------|-------------|-------------------------------|
| **green** | Bande verte | Pénètre l'eau, détecte turbidité |
| **nir08** | Proche infrarouge | Distingue eau/végétation |
| **swir16** | Infrarouge ondes courtes 1 | Humidité du sol |
| **swir22** | Infrarouge ondes courtes 2 | Minéraux, types de sol |
| **NDMI** | Indice d'humidité | Stress hydrique environnant |
| **MNDWI** | Indice d'eau | Présence/étendue de l'eau |

⚠️ **Temps d'exécution** : L'extraction complète (9319 points) peut prendre plusieurs heures. Pour tester, vous pouvez d'abord extraire un sous-ensemble.

In [10]:
# Charger les données de qualité d'eau (nos sites de mesure)
Water_Quality_df = pd.read_csv("../data/raw/water_quality_training_dataset.csv")

print(f"Nombre de sites : {len(Water_Quality_df)}")
print(f"Colonnes : {list(Water_Quality_df.columns)}")
display(Water_Quality_df.head())

Nombre de sites : 9319
Colonnes : ['Latitude', 'Longitude', 'Sample Date', 'Total Alkalinity', 'Electrical Conductance', 'Dissolved Reactive Phosphorus']


,Latitude,Longitude,Sample Date,Total Alkalinity,Electrical Conductance,Dissolved Reactive Phosphorus
0,-28.760833,17.730278,02-01-2011,128.912,555.0,10.0
1,-26.861111,28.884722,03-01-2011,74.720,162.9,163.0
2,-26.450000,28.085833,03-01-2011,89.254,573.0,80.0
3,-27.671111,27.236944,03-01-2011,82.000,203.6,101.0
4,-27.356667,27.286389,03-01-2011,56.100,145.1,151.0


In [11]:
# =============================================================================
# EXTRACTION DES DONNÉES LANDSAT - TRAINING
# =============================================================================

# Option 1 : Extraire tous les points (LONG - plusieurs heures)
# df_to_process = Water_Quality_df

# Option 2 : Extraire un sous-ensemble pour tester (RAPIDE - quelques minutes)
df_to_process = Water_Quality_df.head(200)  # Modifier ce nombre selon vos besoins

print(f"Extraction pour {len(df_to_process)} points...")
print(f"Nombre de workers parallèles : {N_WORKERS}")
print("="*60)

# Extraire les valeurs Landsat
landsat_features = extract_landsat_batch(df_to_process, n_workers=N_WORKERS)

# Calculer les indices spectraux
landsat_features = compute_spectral_indices(landsat_features)

# Renommer les colonnes pour cohérence
landsat_features = landsat_features.rename(columns={'nir08': 'nir'})

print("\n" + "="*60)
print("EXTRACTION TERMINÉE !")
print("="*60)

Extraction pour 200 points...
Nombre de workers parallèles : 4


Extraction Landsat: 100%|██████████| 200/200 [04:31<00:00,  1.36s/it]


EXTRACTION TERMINÉE !


In [12]:
landsat_features.to_csv("landsat_features_200_premieres_lignes")

In [ ]:
# =============================================================================
# CRÉER LE FICHIER CSV FINAL - TRAINING
# =============================================================================

# Ajouter les coordonnées et dates
landsat_train_df = pd.DataFrame({
    'Latitude': df_to_process['Latitude'].values,
    'Longitude': df_to_process['Longitude'].values,
    'Sample Date': df_to_process['Sample Date'].values,
})

# Ajouter les features Landsat
for col in ['nir', 'green', 'swir16', 'swir22', 'NDMI', 'MNDWI']:
    landsat_train_df[col] = landsat_features[col].values

# Sauvegarder en CSV
output_path = os.path.join(OUTPUT_DIR, 'landsat_features_training.csv')
landsat_train_df.to_csv(output_path, index=False)

print(f"Fichier créé : {output_path}")

In [ ]:
# Aperçu du fichier créé
print("Aperçu des données extraites :")
print(f"- Lignes : {len(landsat_train_df)}")
print(f"- Colonnes : {list(landsat_train_df.columns)}")
print(f"\nStatistiques des features Landsat :")
print(landsat_train_df[['nir', 'green', 'swir16', 'swir22', 'NDMI', 'MNDWI']].describe().round(4))
print(f"\nValeurs manquantes :")
print(landsat_train_df.isna().sum())
display(landsat_train_df.head())

In [ ]:
# Sauvegarder dans /tmp et uploader vers Snowflake (uniquement si le notebook tourne dans Snowflake)
# Décommenter ces lignes pour exécuter dans Snowflake

# landsat_train_df.to_csv("/tmp/landsat_features_training.csv", index=False)
# session.sql(f"""
#     PUT file:///tmp/landsat_features_training.csv
#     snow://workspace/USER$.PUBLIC.DEFAULT$/versions/live/
#     AUTO_COMPRESS=FALSE
#     OVERWRITE=TRUE
# """).collect()
# print("Fichier uploadé vers Snowflake !")

print("Fichier 'landsat_features_training.csv' sauvegardé localement !")

---

## Étape 4 : Extraction pour les données de validation (test)

Même processus, mais pour les données de **validation** (le fichier `submission_template.csv`).

C'est ce fichier qu'on utilisera pour faire nos prédictions finales.

In [ ]:
# Charger le template de soumission (sites de validation)
Validation_df = pd.read_csv('../data/raw/submission_template.csv')

print(f"Nombre de sites de validation : {len(Validation_df)}")
display(Validation_df.head())

In [ ]:
# =============================================================================
# EXTRACTION DES DONNÉES LANDSAT - VALIDATION
# =============================================================================

print(f"Extraction pour {len(Validation_df)} points de validation...")
print(f"Nombre de workers parallèles : {N_WORKERS}")
print("="*60)

# Extraire les valeurs Landsat
landsat_val_features = extract_landsat_batch(Validation_df, n_workers=N_WORKERS)

# Calculer les indices spectraux
landsat_val_features = compute_spectral_indices(landsat_val_features)

# Renommer les colonnes pour cohérence
landsat_val_features = landsat_val_features.rename(columns={'nir08': 'nir'})

print("\n" + "="*60)
print("EXTRACTION VALIDATION TERMINÉE !")
print("="*60)

In [ ]:
# =============================================================================
# CRÉER LE FICHIER CSV FINAL - VALIDATION
# =============================================================================

# Ajouter les coordonnées et dates
landsat_val_df = pd.DataFrame({
    'Latitude': Validation_df['Latitude'].values,
    'Longitude': Validation_df['Longitude'].values,
    'Sample Date': Validation_df['Sample Date'].values,
})

# Ajouter les features Landsat
for col in ['nir', 'green', 'swir16', 'swir22', 'NDMI', 'MNDWI']:
    landsat_val_df[col] = landsat_val_features[col].values

# Sauvegarder en CSV
output_path = os.path.join(OUTPUT_DIR, 'landsat_features_validation.csv')
landsat_val_df.to_csv(output_path, index=False)

print(f"Fichier créé : {output_path}")

In [ ]:
# Aperçu des données de validation
print(f"Données de validation : {len(landsat_val_df)} lignes")
print(f"Colonnes : {list(landsat_val_df.columns)}")
print(f"\nStatistiques des features Landsat :")
print(landsat_val_df[['nir', 'green', 'swir16', 'swir22', 'NDMI', 'MNDWI']].describe().round(4))
print(f"\nValeurs manquantes :")
print(landsat_val_df.isna().sum())
display(landsat_val_df.head())

In [ ]:
# Sauvegarder dans /tmp et uploader vers Snowflake (uniquement si le notebook tourne dans Snowflake)
# Décommenter ces lignes pour exécuter dans Snowflake

# landsat_val_df.to_csv("/tmp/landsat_features_validation.csv", index=False)
# session.sql(f"""
#     PUT file:///tmp/landsat_features_validation.csv
#     snow://workspace/USER$.PUBLIC.DEFAULT$/versions/live/
#     AUTO_COMPRESS=FALSE
#     OVERWRITE=TRUE
# """).collect()
# print("Fichier uploadé vers Snowflake !")

print("Fichier 'landsat_features_validation.csv' sauvegardé localement !")

---

## Résumé

**Ce qu'on a fait :**
1. Connecté à Microsoft Planetary Computer
2. Pour chaque site de mesure :
   - Trouvé l'image Landsat la plus proche (nuages < 10%)
   - Extrait les valeurs spectrales (buffer 100m, médiane)
3. Calculé les indices spectraux (NDMI, MNDWI)
4. Créé 2 fichiers CSV avec toutes les features

**Variables extraites :**

| Variable | Description |
|----------|-------------|
| nir | Proche infrarouge (bande 5) |
| green | Vert (bande 3) |
| swir16 | Infrarouge ondes courtes 1 (bande 6) |
| swir22 | Infrarouge ondes courtes 2 (bande 7) |
| NDMI | Normalized Difference Moisture Index |
| MNDWI | Modified Normalized Difference Water Index |

**Fichiers créés :**

| Fichier | Lignes | Colonnes |
|---------|--------|----------|
| landsat_features_training.csv | ~9300 | Latitude, Longitude, Sample Date + 6 variables |
| landsat_features_validation.csv | 200 | Latitude, Longitude, Sample Date + 6 variables |

**Note sur les valeurs manquantes :**

Certains points peuvent avoir des valeurs NaN si :
- Aucune image Landsat n'est disponible pour cette date/localisation
- Toutes les images disponibles ont trop de nuages (>10%)
- Le pixel est invalide (erreur de capteur)